# Importing Eye- and pupil data

In [ ]:
import sys, os
sys.path.insert(0,"..") # this is not needed if you have installed pypillometry
import pypillometry as pp
print(pp.__package_path__) # check that correct version is being used
#import pandas as pd
import numpy as np
#import pylab as plt

/Users/mmi041/miniconda3/envs/pypil/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/mmi041/Dropbox/work/projects/pupil/pypillometry/pypillometry


In [ ]:
pp.PupilData.from_file("../data/example_shortblinks.pd")

In [ ]:
obj = pp.EyeData(1)
obj.summary()

In [ ]:
from importlib import reload
pp = reload(pp)

In [30]:

from collections.abc import MutableMapping
class EyeDataDict(MutableMapping):
    """A dictionary that contains ndarrays of equal length"""

    def __init__(self, *args, **kwargs):
        self.data = dict()
        self.length=0
        self.update(dict(*args, **kwargs))  # use the free update to set keys

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        if not isinstance(value, np.ndarray):
            raise ValueError("Value must be numpy.ndarray")
        if len(value.shape)>1:
            raise ValueError("Array must be 1-dimensional")
        if self.length==0:
            self.length=value.shape[0]
        if value.shape[0]!=self.length:
            raise ValueError("Array must have same length as existing arrays")
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self):
        return iter(self.data)
    
    def __len__(self):
        return self.length
    def __repr__(self) -> str:
        r="EyeDataDict(vars=%i,n=%i): \n"%(len(self.data), self.length)
        for k,v in self.data.items():
            r+="  %s (%s): "%(k,v.dtype)
            r+=",".join(v[0:(min(5,self.length))].astype(str))


In [42]:
a=EyeDataDict(b=np.array([1,2]))
v=np.array([1,2,3])
l=3
",".join(v[0:(min(5,l))].astype(str).tolist())

#len(a)
#a

'1,2,3'

In [38]:
len(a["b"])

2

In [39]:
a

TypeError: __repr__ returned non-string (type NoneType)